In [12]:
# ignore warning
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv("data/ner_dataset.csv", encoding="utf-8")
data = data.fillna(method="ffill")
data.tail(10)

,Sentence #,Word,Tag
369382,Sentence: 16858,vùng,O
369383,Sentence: 16858,chôn_cất,O
369384,Sentence: 16858,người_thân,O
369385,Sentence: 16858,khi,O
369386,Sentence: 16858,qua_đời,O
369387,Sentence: 16858,",",O
369388,Sentence: 16858,gọi,O
369389,Sentence: 16858,là,O
369390,Sentence: 16858,nhị_tì,O
369391,Sentence: 16858,...,O


In [5]:
words = list(set(data["Word"].values))
n_words = len(words); n_words

19151

In [6]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
    
    def get_next(self):
        try:
            s = self.data[self.data["Sentence #"] == "Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s["Word"].values.tolist(), s["Tag"].values.tolist()    
        except:
            self.empty = True
            return None, None

In [8]:
getter = SentenceGetter(data)
sent, tag = getter.get_next()
print(sent); print(tag)

['Đó', 'là', 'con', 'đường', 'biển', 'ngắn', 'nhất', 'để', 'đi', 'từ', 'Ấn_Độ_Dương', 'sang', 'Thái_Bình_Dương', ',', 'chiếm', 'đến', 'lượng', 'hàng_hoá', 'lưu_thông', 'đường_biển', 'của', 'thế_giới', ',', 'đó', 'là', 'hải_trình', 'lớn', 'nhất', 'từ', 'tây', 'sang', 'đông', 'với', '50.000', 'lượt', 'tàu_bè', 'qua_lại', 'mỗi', 'năm', '...']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'LOCATION', 'O', 'LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [9]:
from sklearn.base import BaseEstimator, TransformerMixin


class MemoryTagger(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y):
        '''
        Expects a list of words as X and a list of tags as y.
        '''
        voc = {} # là một dictionary ánh xạ: word -> từ điển đếm của những tag cửa word đó
        self.tags = []
        for x, t in zip(X, y):
            if t not in self.tags:
                # nếu list tag không chứa tag, thêm tag vào list
                self.tags.append(t)
            if x in voc:
                if t in voc[x]:
                    voc[x][t] += 1
                else:
                    voc[x][t] = 1
            else:
                voc[x] = {t: 1}
        self.memory = {}
        for k, d in voc.items():
            self.memory[k] = max(d, key=d.get) # memory của một word là tag phổ biến nhất của word đó
    
    def predict(self, X, y=None):
        '''
        Predict the the tag from memory. If word is unknown, predict 'O'.
        '''
        return [self.memory.get(x, 'O') for x in X]

In [10]:
# Kiểm tra tag
tagger = MemoryTagger()
tagger.fit(sent, tag)
print(tagger.predict(sent))
tagger.tags

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'LOCATION', 'O', 'LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


['O', 'LOCATION']

In [13]:
from sklearn.cross_validation import cross_val_predict
from sklearn.metrics import classification_report

words = data["Word"].values.tolist()
tags = data["Tag"].values.tolist()
pred = cross_val_predict(estimator=MemoryTagger(), X=words, y=tags, cv=5)

report = classification_report(y_pred=pred, y_true=tags)
print(report)

              precision    recall  f1-score   support

    LOCATION       0.75      0.53      0.62      9028
        MISC       0.43      0.13      0.20       561
           O       0.98      1.00      0.99    345533
ORGANIZATION       0.64      0.33      0.43      3268
      PERSON       0.88      0.73      0.80     11002

 avg / total       0.96      0.97      0.97    369392

